# We will call our LLM in this notebook

In [32]:
# !pip install openai -qU

In [33]:
import pandas as pd
import numpy as np
import os
from pathlib import Path
import openai
from openai import OpenAI
from dotenv import load_dotenv

dir_path = Path.cwd()
data_path = (dir_path / "data").resolve()
temp_path = (dir_path / "temp").resolve()
env_path = (dir_path / ".env").resolve()

load_dotenv(env_path)

pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option("display.max_rows", 40)

openai.api_key = os.environ.get("OPENAI_API_KEY")  # beta.openai.com/login/

In [34]:
# bedrock.list_foundation_models()

## GPT 4

In [35]:
# Let get our prompt file

# Specify the file path
file_path = 'prompts.npy'

# To load
# Load the DataFrame back from the binary file
prompt_df = np.load(file_path, allow_pickle=True)
prompt_df = pd.DataFrame(prompt_df, columns=['member_id', 'prompt'])
prompt_df

,member_id,prompt
0,1TA9YY3MK99,You are the world's foremost investigator into...
1,5E73XN8XG05,You are the world's foremost investigator into...
2,5KN2RV8RX93,You are the world's foremost investigator into...


In [36]:
def create_prompt(prompt):
    # ChatGPT
    chat_prompt = [
        {
            "role": "user", #prompt created by chat.openai.com based on prompting.
            "content": f"{prompt}"
        }
    ]

    return chat_prompt

chat_model = "gpt-4-0613" # "gpt-3.5-turbo-0613" #

In [37]:
import random
# Get a list of unique member IDs
unique_member_ids = prompt_df['member_id'].unique()

# Randomly select a member ID from the list
random_member_id = random.choice(unique_member_ids)
print(random_member_id)
for index, row in prompt_df[prompt_df['member_id']==random_member_id].iterrows():
    # Extract values from the DataFrame row
    prompt = row['prompt']
    prompt = create_prompt(prompt)
    print(prompt)

1TA9YY3MK99
[{'role': 'user', 'content': "You are the world's foremost investigator into fraud, waste, and abuse in the dental insurance industry.\nHere are our definitions for FWA:\n###\nFraud: Fraud hinges on intent: healthcare workers knowingly working outside of accepted standards for their own financial gain. Some common fraud examples include billing for services not rendered, misrepresenting dates of service, diagnosing unnecessary or incorrect treatment, or misrepresenting the identity of the patient. While these aren’t limited to dentistry, spotting nefarious intent within inappropriate claims must be done right away before unwarranted payment escalates.\nAbuse: Abuse typically involves acts that are inconsistent with identified acceptable and legitimate practices. This can result in additional and unnecessary costs to the payer, as well as possible injury to patients, who may suffer from a lack of needed care, incorrect care, or deficient care as a result. Over-coding procedu

In [38]:
def generate_reply(prompt):
    
    try:
        client = OpenAI()

        stream = client.chat.completions.create(
            model="gpt-4",
            messages=prompt,
            temperature=0,
            # max_tokens=400,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0,
            stop=None,
            stream=True,
        )
        for chunk in stream:
            print(chunk.choices[0].delta.content or "", end="")
    except Exception as e:
        print(e)
        return "Sorry. Seems like there's an issue here with generating a reply to your question. Please try again."

In [39]:
generate_reply(prompt)

Based on the provided information, there are several potential issues with the claims submitted by Dr. Smith for patient Evaleen Gaitung. 

1. Line 1: The D0150 procedure code for a comprehensive oral evaluation is only allowed once per lifetime, per dentist/provider. It is not covered when performed on the same date, same provider as D4355. However, in this case, it appears that both D0150 and D4355 were performed on the same day by the same provider. This could be a potential case of Fraud or Abuse as the provider may be knowingly billing for services not allowed by the policy. Severity: High

2. Line 5: The D4341 procedure code for periodontal scaling and root planning is only allowed once per quadrant per twenty-four (24) month period, for patients aged eighteen (18) or older. It is not clear from the provided information if this procedure was necessary or if it was performed more than once in the specified period. If it was unnecessary or performed more than allowed, this could be